# Example notebook

Here we get the Article classes for all given bio.tools tools.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from bh24_literature_mining.utils import load_biotools_pub
from bh24_literature_mining.biotools import Tool_entry

tools = load_biotools_pub("../biotoolspub/biotoolspub.tsv")

# Initialize set to track unique names and a list to store Biotool objects
tools_lower = set()
unique_biotools = []

# Iterate over the rows in the DataFrame
for _, row in tools.iterrows():
    name = row["name"]
    biotools_id = row["biotoolsID"]
    name_lower = name.lower()

    # Only add unique tools based on name
    if name_lower not in tools_lower:
        tools_lower.add(name_lower)  # Add to the set to track uniqueness
        # Create a Biotool object and add it to the list
        biotool = Tool_entry(biotool_id=biotools_id, name=name)
        unique_biotools.append(biotool)


print(len(unique_biotools))

## Example on how to use europepmc_api

In [ ]:
from pathlib import Path
from bh24_literature_mining.europepmc_api import EuropePMCClient, write_tool_mentions_to_file
from bh24_literature_mining.europepmc_api import identify_tool_mentions_in_sentences
import pandas as pd

results_list = []

client = EuropePMCClient()

for tool in unique_biotools[:100]:
    # Call bio.tools query and get a list of Article objects
    
    biotools_articles = client.search_mentions(tool.name)

    if len(biotools_articles) == 0:
        continue
    first_article = biotools_articles[0]

    relevant_parahraphs = client.get_relevant_paragraphs(first_article.pmcid, tool.name)
    if len(relevant_parahraphs) == 0:
        print("No relevant paragraphs found", tool.name)
        continue
    
    tool.get_topics()


    result = identify_tool_mentions_in_sentences(first_article.pmcid, tool, relevant_parahraphs)
    results_list.extend(result)


In [ ]:

result_df = pd.DataFrame(results_list, columns=["PMCID", "Sentence", "NER_Tags"])
result_df = result_df.explode("NER_Tags").drop_duplicates()

# p = Path(__file__).resolve().parents[2]
# print(p)
result_df.to_csv("/Users/vedran/Desktop/tmp.csv", index=False)



In [ ]:
len(results_list)

In [ ]:
result_df.sample(1)